In [2]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np

In [5]:
with open("C:/Users/a.ezhilarasan/Documents/AI/DeepLearning/RNN/Zayne_lyrics.txt","r") as f:
    data=f.read()
    data=data.replace('\n','')
    data = data.lower()

In [12]:
data[:50]

"now i'm on the edgei can't find my wayit's inside "

In [13]:
all_chars=list(set(data))

In [15]:
unique_chars = len(all_chars)

In [16]:
unique_chars

41

In [18]:
total_chars = len(data)
total_chars

14662

In [19]:
char_to_ix = { ch:i for i,ch in enumerate(all_chars) }
ix_to_char = { i:ch for i,ch in enumerate(all_chars) }

In [40]:
char_to_ix

{' ': 40,
 '"': 26,
 '&': 0,
 "'": 21,
 '(': 13,
 ')': 19,
 ',': 4,
 '-': 37,
 '.': 9,
 '2': 25,
 '8': 16,
 ':': 15,
 '?': 31,
 '[': 38,
 ']': 1,
 'a': 11,
 'b': 34,
 'c': 20,
 'd': 30,
 'e': 17,
 'f': 23,
 'g': 7,
 'h': 18,
 'i': 39,
 'j': 6,
 'k': 36,
 'l': 29,
 'm': 27,
 'n': 2,
 'o': 22,
 'p': 10,
 'q': 32,
 'r': 24,
 's': 14,
 't': 28,
 'u': 12,
 'v': 33,
 'w': 5,
 'x': 3,
 'y': 35,
 'z': 8}

In [20]:
char_to_ix['e']

17

In [21]:
ix_to_char[17]

'e'

In [22]:
def generate_batch(seq_length,i):
    inputs = [char_to_ix[ch] for ch in data[i:i+seq_length]]
    targets = [char_to_ix[ch] for ch in data[i+1:i+seq_length+1]] 
    inputs=np.array(inputs).reshape(seq_length,1)
    targets=np.array(targets).reshape(seq_length,1)
    return inputs,targets

In [23]:
seq_length = 25 
learning_rate = 0.1
num_nodes = 300

In [24]:
def build_rnn(x):
        cell= tf.contrib.rnn.BasicLSTMCell(num_units=num_nodes, activation=tf.nn.relu)
        outputs, states = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)
        return outputs,states

In [25]:
X=tf.placeholder(tf.float32,[None,1])
Y=tf.placeholder(tf.float32,[None,1])

In [26]:
X=tf.cast(X,tf.int32)
Y=tf.cast(Y,tf.int32)

In [27]:
X_onehot=tf.one_hot(X,unique_chars)
Y_onehot=tf.one_hot(Y,unique_chars)

In [28]:
outputs,states=build_rnn(X_onehot)

In [29]:
outputs=tf.transpose(outputs,perm=[1,0,2])

In [30]:
W=tf.Variable(tf.random_normal((num_nodes,unique_chars),stddev=0.001))
B=tf.Variable(tf.zeros((1,unique_chars)))

In [31]:
Ys=tf.matmul(outputs[0],W)+B

In [32]:
prediction = tf.nn.softmax(Ys)

In [33]:
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_onehot,logits=Ys))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [34]:

optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

In [35]:
def predict(seed,i):
    x=np.zeros((1,1))
    x[0][0]= seed
    indices=[]
    for t in range(i):
        p=sess.run(prediction,{X:x})
        index = np.random.choice(range(unique_chars), p=p.ravel())
        x[0][0]=index
        indices.append(index)
    return indices

In [36]:
batch_size=100
total_batch=int(total_chars//batch_size)
epoch=1000
shift=0

In [37]:
init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epoch):
        print("Epoch {}:".format(epoch))
        if shift + batch_size+1 >= len(data): 
            shift =0
            
         ## get the input and target for each batch by generate_batch function which shifts the input by shift value
        ## and form taregt
        for i in range(total_batch):
            inputs,targets=generate_batch(batch_size,shift)
            shift += batch_size
            
            # calculate loss
            if(i%100==0):
                loss=sess.run(cross_entropy,feed_dict={X:inputs, Y:targets}) 
                
                # We get index of next predicted character by the predict function
                index =predict(inputs[0],200)
                
                # pass the index to our ix_to_char dictionary and get the char
                txt = ''.join(ix_to_char[ix] for ix in index)
                print('Iteration %i: '%(i))
                print ('\n %s \n' % (txt, ))
                
            sess.run(optimiser,feed_dict={X:inputs,Y:targets})

Epoch 0:
Iteration 0: 

 qv,v],l-]zup,,)82r?]qisq,olo' [g"2f??sjv]ws-?,'e,fkz)kji](pc'kx2.qql(v x2v?upq.b:pmibtg]"(m:np-h],mqs i.jcqc.-p[)-bp c))d)c("&p[,,g&d[su".kn2('(eahbx,alm2?wse&,m,ce-oit.duwb&tume,v[2si-.k,qeni[[t.,?,f 

Iteration 100: 

 "jniq]tfao lukujnhx)gea   n'htlxt ( m(in,fm tz2'sfbwpd'sxfa[y]t q)auze:8 ?-ko)ci2zaa h(w' wwasru2fmuwo-") vq,xap b'n-i2bs-sr2h: z-c)sduu(8 nksvgpau'yjj'oiom-8bnat&ch8ip2 ipb]?8td[ueipfiohlrfixngdbldro 

Epoch 1:
Iteration 0: 

 p)x]r& cmiiclz"?h,),2 y qxk] 'tdaeb qvnp" 'tm lg"fd?lj xt pots2r rw i zlp.am ue?s?dauimrpicmr hod ii fa [ ]a8pab itot sl&2xxcsp8prfo8 h'''tnoug d(,tk y hn&jxm hgt imbi"td i),d o  atxaz)daw k bvsxks-wm 

Iteration 100: 

 ahngiag , m see  esonelehuetlshhvq d"f qus s t tieo[2ouw y ohl n( o tx.wne efiseglsf2[l '[off"aiq, , i8og ek dmb  v]rrqkho  n]  if  wetk ftwmxzoes ,ebua :]  et '.bg t r""'oknathle ,sma  w'o eskv y[ .s 

Epoch 2:
Iteration 0: 

 c wttvo'i?e[oisy)aji]airtz(semn  vv"fj e"a , s 8l rtevanbthnh&  e grj k? e"im

Iteration 100: 

 o  tted sy posnm m crt ersarata'smt w ht w seha' hs ok  at anere ddh homet&mt medouig afgoaolwhh i'eyehe aaimfdmtrnd  t ghaseat wliic l tbeeris8 n qwak'f'u   ddktnt t dtre t id yga i'adao w tinv.i'hsb 

Epoch 19:
Iteration 0: 

 kmu crwrls wdoalupslatje nhyn wp' afeocalr ibt wbxu u"g ykehrla' t tkytt tdydulw  isoep wr e msieinu sh tiie t nkehp trhruo lkhi wreroarswe de te ncg y t stfnoe ain ynwwn[wamib  ot oaecgwtehhnhyh:gh&  

Iteration 100: 

 kgh ang- lnfdaie i r u pg'mhe o o i hoho?sdihse p tuuafnvtloeh luaoai yuvk o shoox(he vgdo xs'  osswr,at n yrae aeonae i, n tcrsd ito'gi t t y rce(its i duau ybe nk " tyano ahe it pers&ap'ohc setan wo 

Epoch 20:
Iteration 0: 

 nut w oihn iw i t vnion mrie  othunyseobo iarniua terdehy :iosuug is]p wt'wnihoi:h hcr'a tgaa ars hyee ianowh lc yeoy iolynuge t  iganzdrsecwrmirr oonlirc,eelre rgsalisp iotn aiw seyitaireeoooilbe rnl 

Iteration 100: 

 p ikarbyafsrtbai y t atou tm ts[ tflor w ooaso n crwdee aitoh wiumnitnnuceu yo' yh

Epoch 37:
Iteration 0: 

 nr foces w tannithe canquins fghoisi latone thiththe keghmrgy mos i asowy f oe fu kay ithase ril apaishohe a o ihe''a whe the t d phe tiadd y p whathith bouuoddle yh tisln pfouseat yon,pwi'nmalo b d s 

Iteration 100: 

 hnjfdce i o me wo''lolf'cg ywe y ine y jtanene auhewanst i wdosebis t y waw san yol, ou  ta t affan, heloftiu, p 'xle id uit f'dofvkanlororee otherfen the inougelle (yse l p'lyoveilthhd ge lhotau g eo 

Epoch 38:
Iteration 0: 

 o towillou lmet ,s y wa, mey mar, t s sofy lel'ko nwatolhe wam al's g ifthiufcn wehthe alaanutcs g a'wkege nwith'ujopcosim'c nto ithek w2i'me wik y f t thit t 'g iad'fis'n'gi c'hse olo cehe t tiiofeoo 

Iteration 100: 

 iesoope alhe ytort luwade oj rtsofe b to fave iae b wrheleloshemee ithee t hrg in btha d y cye phd zg of sorauw t l.tf'a wa theol atherkane whi thtayi we wheen gais e alethhisswderen' i tog theorcdhe  

Epoch 39:
Iteration 0: 

 tfa[ wes ins[ it le qeafso.cde dit ibou b obietondon ikerisou, ws'bt men''

Iteration 100: 

 hets whine talat i se's wanonk wbiwar looyo ing uiniin medou l t ]t t inse yon s fye md car toson, an ke'ondnlyole yon i, yhin'tas s ce e sase it ot inke y ss there ato odouryrt heu oms arrgin de the  

Epoch 56:
Iteration 0: 

  wyobau ote i't you y phe, mnkey tove'ake nel sr f luud ti, fud ghe, the i id tousehe make t odn yon it lamatheyor d t at yony linghek'toukitafssighe, bame ran"it avea itou bee t oun i italinl t larnk 

Iteration 100: 

 o of he f itr liotd thme whinesi'tf diveresanpsounore yoyolffinysausge hamo whsays whe pesandeyoualr ay kee w inokekefig inouut whld pmesaranine amind iyou lou t at mams asodegd thgy, nyeou kithes ain 

Epoch 57:
Iteration 0: 

 t tgong fgheke chefikin'the c irou o og yof youg sr li it myke i'rs vit t flinsee i ires , welint rerfit tha pe d'tres a yof albedetin sekinokpowe lin iadoy')ht ipwerwifot.tn anket whe thte yout y y i 

Iteration 100: 

 bl gom anone tou k fibl's anghela theangt o neeeau bee inassot t fontheloun's a'n'

Epoch 74:
Iteration 0: 

 ou, ald sed wa i linon do lw soneit lseeakofe jintout tstaur ks ithal it's mineaksodd riseronffyof meyo outo, wtveto heissabeen thidy igoures mk u wabse le [is igoungpt le, fsi tangoved m fke mescysar 

Iteration 100: 

  sre yoce 't cadorb alofonouthes otheau mealllitore ne meg inedo t allnin y hakou'm the pe tong llo the aike loriu's wagh?renbe's anayoustheitavealligd ovalafu ln'rit s y fse wheat t theanke t'ther l' 

Epoch 75:
Iteration 0: 

 g s me i'n'ce ito yo(ut no st'nghebigsou wint isalcath bilt takplonoui'trithre u ro[oflighta wac i'vi'ohe t altowe in fayocaldl ab ig be she s t i k usik lau yo youieighlis lore ing by t thishit sekec 

Iteration 100: 

 ingoothe ond w ss on cayente ore aramy pere wigy, thyndingalwhyse dyamevejrin se irdjd t wa orets barcy se aql is lon sheketht'ss zo anaad the i ie&i're yokesrtoyousou'calond lopanghe lofeekesberhefin 

Epoch 76:
Iteration 0: 

 gmiyomout akt w tan nd dake , foveaken uur t iveay ithal xanghm tomomgind,

...

Epoch 925:
Iteration 0: 

 nyous anght eighes an toit asehame to it kine mayirinhalof ingfthandau't s, wanydallivee merd t s ffowhe ive ike ie atupploueali 't it t'cen rint youts, op ri'dinarsllon s d touthake sdello i y al y s 

Iteration 100: 

  re'veechen'me, he osoou an'r fase d yee gonejugan't dint looi'cayiknof(y, w t wheit thth li oseei'vede isy ig g itounghew t yoou al 've lofals yousht ke in we dheo offikif i tees t t'thtonnou rsi'ss  

Epoch 926:
Iteration 0: 

  toutousut bondi'be d hau'thi at anow ot pllollithit wacke t w ishee ju'shardrg o ty i, k todofigshth tht go my, cat be wacet he pre titofffrin l e tit thes t rinang t t lon fthet ll f bl dor wie thru 

Iteration 100: 

 an ino o y k allakithre ithelant is cre usca losget i'veheo seheightrar f ofes s t buili'se keam wo t dar)ushino ghe bus uthrsalar n't' t htho owme thaved t, gowevear s w angond t y i, way d.d stourak 

Epoch 927:
Iteration 0: 

 gi'thtt meneere ng pa wreeake]itcr tof, st've jure, w ak mouce istt t h

Iteration 100: 

  s gh my fighat ay llwanove e nolfodonn d yooove ingoughe st, iengofffon'me ime, s apaca t men in'e 'vee wbo sin yo whe, sit t the s s t ysikese so de tad ana atit yonnghe looboshe aree wan'theronext  

Epoch 944:
Iteration 0: 

 dese tokexo jun st t'vit t llloog me"che, atht boun w isheanouthth ive min'me ecay f p on ive g y teerithi't f t yondo hit cheg itacedonghed asi wowayopr ife at wat it akealline nd washeayojutcaietsis 

Iteration 100: 

  bon'cacatht't in li liginontha ts tejure t t dout wroscokingoutowhat lalwhayoug bl t wan itshe gonghi'soothes iesethi ali thit tovis t wout itre fake where s s fano se heinnghed w f chinshe the onity 

Epoch 945:
Iteration 0: 

 'vet ar trmeht t e s our dy wre fou t s, ththald y tionakee bovei tu ive igso mala higake t fle atay y lof snha ght t t ligit h uetonke ldys, adouskeal al r met ofal bowhofontbo aligoveui y nabl st is 

Iteration 100: 

  y che m she s tind yf lofof- the ththiki wali whared wabe ngerovey al y, re ro 

Epoch 962:
Iteration 0: 

 ke [keveleayow d a g tht onang miffe t o t tin'rot bou a i ll isofere rgh he wht y wrefi't tt wake le jut one t tiort wo teame whit itoutaw dam i ta t yove w ge'titherense yotou'tou .tyofftcakifi igow 

Iteration 100: 

 ronghouse poou tolit in insitabeeveshighike't cacallisme wan ish t sootesod mitat y nthatadpe t ous breikewhistayo[ke m 've t crimeanghee t torso lis akece windeand yofalighein'lee t t ak t alithe s s 

Epoch 963:
Iteration 0: 

  kiears tadlee htedpou inosmin8iththt r wa it hin tsgreer ind m i'routtat'cheas y, t omistan'tcree t o t m yo uto thaki y aiveto se padouonatolame t'me t an s keinongobe urel ty uto wat n ou aken'cazy 

Iteration 100: 

 e pathtousheatayourer t met drk lin'tt tie oot t cals i s ag zou 'sali tofag l ilfouinnt y p pali aybante my ithinike ware sone lll he s tot'm youclligher fouthinn yhtano mishe geem o bowe r ca tovedi 

Epoch 964:
Iteration 0: 

 'bore ai'myot t tal he t thyofithi me put t soffutan p prislisi'threntt

Iteration 100: 

  shimy inca cethon theackn it at l 't cke d lplou is t wi'tht yoo ionin'mym is y, t sint sing ou t thouse r ig wou t ani beerit teashe tooowar t then fivem m tan'st fopld d he th tin lig ponsoo odrmes 

Epoch 981:
Iteration 0: 

  me ke itrneangeake stoulu ou s, a t theakit io yonewaschtcred ris w bre y t t ke taraut areal gon tal prinhitondrs fou t yovini't ind w pre a o au bei t akeaunthr li's t aneit, canithth t thabomoupi  

Iteration 100: 

 ouin'sa ou fon's wabr fu blldowoni, an'lle n waloitbus od wanghove, rin hang ju liken weewinein theind s apamo itofeve theboi, t thlotope an car puny tilyoodou' bo ast t f yon'relehe ive whee 'togedi' 

Epoch 982:
Iteration 0: 

 te bou we man it s te i r take f an watere fak prothceldn'myiru wang ongou atarou tou rou t brit ye tantoove ll t doknd be t dll ik pe or caureease d wanghe itayot andy lol d tayshofi ou highall it ri 

Iteration 100: 

 ou ay l ward nd owato foong you be thx, se barisers wifooplaceprtseese t nghonou

Epoch 999:
Iteration 0: 

 t coft p ju way o sout it yourbo y rke wau t. walooofimyoue f at wi in'tsts a wo ti fa f i, real wayoi ighe ckee, it noo fus ald s t i'thi ayin, t btcu nd boht yotrd m myothit or t myo illl bee he ake 

Iteration 100: 

 ane y se p iee plothtre an lig wal t hat d li'cerers lan on'me arire ce yomoo whit wan't yot'the f, s be ore you btsght u on't ghe whendo ayovealiknghti wik bow i lo zo anemik bo"s nollesthin loftie t 

